In [8]:
import os
import cv2
import numpy as np
import torch
from torchvision import transforms
from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint
from models.yolo import Model
from PIL import Image, ImageEnhance
import random
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix

torch.serialization.add_safe_globals([Model])

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

weights = torch.load('yolov7-w6-pose.pt', map_location=device, weights_only=False)
model = weights['model']
_ = model.float().eval()

if torch.cuda.is_available():
    model.half().to(device)

# SARAN AI TAPI MALAH TURUN
def augment_image(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(image)
    
    brightness_factor = random.uniform(0.8, 1.2)
    enhancer = ImageEnhance.Brightness(image)
    image = enhancer.enhance(brightness_factor)
    
    contrast_factor = random.uniform(0.8, 1.2)
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(contrast_factor)
    
    image = np.array(image)
    mean = 0
    var = random.uniform(0, 0.005)
    sigma = var ** 0.5
    gaussian = np.random.normal(mean, sigma, image.shape).reshape(image.shape)
    image = image + gaussian * 255
    image = np.clip(image, 0, 255).astype(np.uint8)
    
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image

def apply_clahe(image):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l_clahe = clahe.apply(l)
    lab_clahe = cv2.merge((l_clahe, a, b))
    return cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)

def retinex_deillumination(image):
    
    # Apply Retinex deillumination to the image.
    #  image: Input image (numpy array in BGR format).
    # return: Image with Retinex deillumination applied (numpy array in BGR format).
    
    # Convert the image to LAB color space
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    
    # Apply Gaussian blur to the L channel
    l_blur = cv2.GaussianBlur(l, (0, 0), 3)
    
    # Subtract the blurred L channel from the original L channel
    l_deilluminated = cv2.subtract(l, l_blur)
    
    # Merge the channels back and convert to BGR
    lab_deilluminated = cv2.merge((l_deilluminated, a, b))
    return cv2.cvtColor(lab_deilluminated, cv2.COLOR_LAB2BGR)

def preprocess_image(image, apply_augmentation=True, apply_clahe_flag=True, apply_retinex_flag=True):
    
    # Preprocess the image with augmentation, CLAHE, Retinex deillumination, and resizing.
    #  image: Input image (numpy array in BGR format).

    if apply_augmentation:
        image = augment_image(image)

    if apply_clahe_flag:
        image = apply_clahe(image)

    if apply_retinex_flag:
        image = retinex_deillumination(image)

    # Resize image to 640x480
    image = cv2.resize(image, (640, 480))

    return image

def load_ground_truth(label_path):
    
    # return: Integer ground truth value (0 for 'adl', 1 for 'fall').
    
    with open(label_path, 'r') as file:
        line = file.readline().strip()  # Baca baris pertama dan hapus whitespace

        if not line:  # Cek jika kosong
            return 0  # Default: asumsikan 'adl' jika tidak ada data

        values = line.split()
        if len(values) == 0:  # Jika tetap kosong setelah split
            return 0  # Default: tidak ada deteksi jatuh

        first_value = values[0]

        return int(first_value)  # Konversi ke integer

        
def detect_fall(keypoints, threshold=0.5):
    
    # Detect fall based on keypoints.
    # keypoints: Array of keypoints (17 keypoints, each with x, y, confidence).
    # threshold: Confidence threshold for keypoints.
    # return: True if fall is detected, False otherwise.
    
    # Indices for keypoints (COCO format)
    LEFT_SHOULDER = 5
    RIGHT_SHOULDER = 6
    LEFT_HIP = 11
    RIGHT_HIP = 12
    LEFT_KNEE = 13
    RIGHT_KNEE = 14

    # Get keypoints and confidence scores (IKUT RUMUS DARI MAIN JURNAL)
    left_shoulder = keypoints[LEFT_SHOULDER * 3: (LEFT_SHOULDER + 1) * 3]
    right_shoulder = keypoints[RIGHT_SHOULDER * 3: (RIGHT_SHOULDER + 1) * 3]
    left_hip = keypoints[LEFT_HIP * 3: (LEFT_HIP + 1) * 3]
    right_hip = keypoints[RIGHT_HIP * 3: (RIGHT_HIP + 1) * 3]
    left_knee = keypoints[LEFT_KNEE * 3: (LEFT_KNEE + 1) * 3]
    right_knee = keypoints[RIGHT_KNEE * 3: (RIGHT_KNEE + 1) * 3]

    # Check confidence scores (IKUT RUMUS DARI MAIN JURNAL)
    if (left_shoulder[2] < threshold or right_shoulder[2] < threshold or
        left_hip[2] < threshold or right_hip[2] < threshold or
        left_knee[2] < threshold or right_knee[2] < threshold):
        return False  # Skip if any keypoint is not confident

    # Calculate average y positions (IKUT RUMUS DARI MAIN JURNAL)
    shoulder_y = (left_shoulder[1] + right_shoulder[1]) / 2
    hip_y = (left_hip[1] + right_hip[1]) / 2
    knee_y = (left_knee[1] + right_knee[1]) / 2

    # Check if hip and knee are below shoulders (fall condition)
    return hip_y > shoulder_y and knee_y > shoulder_y


ground_truth_list = []
predictions_list = []

test_images_path = "test/images"
test_labels_path = "test/labels"

for filename in os.listdir(test_images_path):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        image_path = os.path.join(test_images_path, filename)
        label_path = os.path.join(test_labels_path, filename.replace(".jpg", ".txt").replace(".png", ".txt"))
        
        image = cv2.imread(image_path)
        image = preprocess_image(image, apply_augmentation=False, apply_clahe_flag=True, apply_retinex_flag=True)
        
        image_resized = letterbox(image, 960, stride=64, auto=True)[0]
        image_tensor = transforms.ToTensor()(image_resized)
        image_tensor = torch.tensor(np.array([image_tensor.numpy()]))
        
        if torch.cuda.is_available():
            image_tensor = image_tensor.half().to(device)
        
        with torch.no_grad():
            output, _ = model(image_tensor)
            output = non_max_suppression_kpt(output, 0.25, 0.65, nc=model.yaml['nc'], nkpt=model.yaml['nkpt'], kpt_label=True)
            output = output_to_keypoint(output)
        
        fall_detected = 0
        for idx in range(output.shape[0]):
            keypoints = output[idx, 7:].T
            if detect_fall(keypoints):
                fall_detected = 1
                break

        
        predictions_list.append(fall_detected)
        ground_truth_list.append(load_ground_truth(label_path))

#Calculate Metrix
precision = precision_score(ground_truth_list, predictions_list)
recall = recall_score(ground_truth_list, predictions_list)
f1 = f1_score(ground_truth_list, predictions_list)
accuracy = accuracy_score(ground_truth_list, predictions_list)

# Calculate specificity
tn, fp, fn, tp = confusion_matrix(ground_truth_list, predictions_list).ravel()
specificity = tn / (tn + fp)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")
print(f"Accuracy: {accuracy:.2f}")
print(f"Specificity: {specificity:.2f}")

Precision: 0.92
Recall: 0.73
F1-score: 0.81
Accuracy: 0.80
Specificity: 0.90
